In [1]:
import numpy as np
import pandas as pd
import scrapy
import requests
from lxml import etree
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from io import StringIO
from scrapy.crawler import CrawlerProcess
from pathlib import Path
import time
import datetime

# Constants

In [2]:
PATH_DATA = Path.cwd().parent.parent / 'data'
PATH_DATA.exists()

True

In [3]:
PATH_SCRAPED = PATH_DATA / 'raw' / 'scraped'
PATH_SCRAPED.exists()

True

In [4]:
PATH_GENOMIC_FNA = PATH_DATA / 'raw' / 'genomes' \
    / 'archaea' / 'genomic_fna'
PATH_GENOMIC_FNA.exists()

True

In [5]:
PATH_TEMP_GENOMIC_FNA = PATH_DATA / 'raw' / 'genomes' \
    / 'archaea' / 'temp_genomic_fna'
PATH_TEMP_GENOMIC_FNA.exists()

True

# Data Recovery

In [6]:
df_archaea = pd.read_csv(PATH_DATA / 'raw' / \
    'scraped' / 'archaea_name_url.csv')
df_archaea

,name,url
0,ANME-2_cluster_archaeon_HR1,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
1,Acidianus_ambivalens,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
2,Acidianus_brierleyi,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
3,Acidianus_hospitalis,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
4,Acidianus_infernus,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
...,...,...
1232,uncultured_Nitrososphaera_sp.,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
1233,uncultured_archaeon_A07HB70,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
1234,uncultured_archaeon_A07HN63,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...
1235,uncultured_archaeon_A07HR60,https://ftp.ncbi.nlm.nih.gov/genomes/refseq/ar...


In [7]:
urls_archaea_organisms = df_archaea['url'].values
print(len(urls_archaea_organisms))
urls_archaea_organisms

1237


array(['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/',
       ...,
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HN63/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR67/'],
      dtype=object)

# Genome Scrapers

## Requests

In [8]:
# ANME-2_cluster_archaeon_HR1

try:
    request_archaea_0 = requests.get(urls_archaea_organisms[0])
    print(request_archaea_0.status_code)
except Exception as e:
    print(e)

200


In [9]:
soup_archaea_0 = BeautifulSoup(request_archaea_0.text, 'html.parser')
type(soup_archaea_0)

bs4.BeautifulSoup

In [10]:
soup_archaea_0

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">

<html>
<head>
<title>Index of /genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1</title>
</head>
<body>
<h1>Index of /genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1</h1>
<pre>Name                            Last modified      Size  <hr/><a href="/genomes/refseq/archaea/">Parent Directory</a>                                     -   
<a href="all_assembly_versions/">all_assembly_versions/</a>          2024-03-18 00:38    -   
<a href="annotation_hashes.txt">annotation_hashes.txt</a>           2024-03-16 19:37  410   
<a href="assembly_summary_historical.txt">assembly_summary_historical.txt</a> 2024-03-16 19:08  1.1K  
<hr/></pre>
<a href="https://www.hhs.gov/vulnerability-disclosure-policy/index.html">HHS Vulnerability Disclosure</a>
</body></html>

In [11]:
result_set_archaea_0 = soup_archaea_0.find('pre').find_all('a')
result_set_archaea_0

[<a href="/genomes/refseq/archaea/">Parent Directory</a>,
 <a href="all_assembly_versions/">all_assembly_versions/</a>,
 <a href="annotation_hashes.txt">annotation_hashes.txt</a>,
 <a href="assembly_summary_historical.txt">assembly_summary_historical.txt</a>]

In [12]:
items = []
for i in result_set_archaea_0[1:]:
    items.append( i.get('href') )
items

['all_assembly_versions/',
 'annotation_hashes.txt',
 'assembly_summary_historical.txt']

In [13]:
urls_archaea_0 = []
for i in result_set_archaea_0[1:]:
    urls_archaea_0.append( urljoin( request_archaea_0.url, i.get('href') ) )
urls_archaea_0

['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/all_assembly_versions/',
 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/annotation_hashes.txt',
 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/assembly_summary_historical.txt']

In [14]:
# aav := all_assembly_versions
url_aav_archaea_0 = [url for url \
    in urls_archaea_0 if 'all_assembly_versions' in url]
print(url_aav_archaea_0)

try:
    request_aav_archaea_0 =  requests.get(urls_archaea_0[0])
    print(request_archaea_0.status_code)
except Exception as e:
    print(e)

['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/all_assembly_versions/']
200


In [15]:
urls_archaea_organisms[:5]

array(['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/',
       'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_infernus/'],
      dtype=object)

In [16]:
for url in urls_archaea_organisms[:5]:
    print(urljoin(url, 'all_assembly_versions/'))

https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/all_assembly_versions/
https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/all_assembly_versions/
https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/all_assembly_versions/
https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/all_assembly_versions/
https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_infernus/all_assembly_versions/


In [17]:
url_aav_ok = 0
url_aav_not_ok = 0
for url in urls_archaea_organisms[:5]:
    try:
        response = requests.get(urljoin(url, 'all_assembly_versions/'))
        print( response.url )
        # access elements of the response.text using xpath


        # Assuming 'response' is the requests response object
        html_parser = etree.HTMLParser()
        parsed_html = etree.HTML(response.text, parser=html_parser)

        # Use XPath expressions to select elements
        elements = parsed_html.xpath('//pre/a')  
        # print(elements)

        urls_aav = [urljoin(url, "all_assembly_versions/") for url in urls_archaea_organisms[:5]]
        print("urls_aav", urls_aav)

        # Process the selected elements
        for element in elements:
            # Access element properties or extract text
            print("-----ELEMENT-----")
            property_value = element.get('href')  # Replace 'property_name' with the desired property name
            print("href: ", property_value)
            element_text = element.text
            print(element_text)
            # Perform further processing on the selected elements
        


        url_aav_ok += 1
        # print(request.status_code)
    except Exception as e:
        print(e)
        url_aav_not_ok += 1

# last runtime: 14m 11.9s

print(url_aav_ok, url_aav_not_ok)

https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/all_assembly_versions/
urls_aav ['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/all_assembly_versions/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/all_assembly_versions/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/all_assembly_versions/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/all_assembly_versions/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_infernus/all_assembly_versions/']
-----ELEMENT-----
href:  /genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/
Parent Directory
-----ELEMENT-----
href:  suppressed/
suppressed/
https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/all_assembly_versions/
urls_aav ['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/all_assembly_versions/', 'https://ftp.ncbi.nlm.nih.gov/ge

## Archaea

### Item and Custom Pipeline

In [18]:
from scrapy.pipelines.files import FilesPipeline
from urllib.parse import urlparse

In [19]:
class genomeNameItem(scrapy.Item):
    file_urls = scrapy.Field()
    genome_file_name = scrapy.Field()

In [20]:
class CustomFilesPipeline(FilesPipeline):
    def get_media_requests(self, item, info):
        # This method is called for each item and must return the Request objects to download.
        # Here, we customize it to use our item's `file_urls` field.
        return [scrapy.Request(x, meta={'genome_file_name': item['genome_file_name']}) for x in item.get('file_urls', [])]

    def file_path(self, request, response=None, info=None, *, item=None):
        # This method determines the path (including the filename) where the file will be saved.
        # Here, we use the filename from the request's meta, which we set in get_media_requests.
        return request.meta['genome_file_name']
    
    # def file_path(self, request, response=None, info=None, *, item=None):
    #     # Extract the desired filename from the item
    #     # This assumes you have a 'genome_file_name' field in your item
    #     filename = item.get('genome_file_name')
    #     # Return the filename
    #     return filename


### Spider

In [21]:
class SpiderGenomesArchaea(scrapy.Spider):
    name = 'spider_genomes'
    allowed_domains = ['ftp.ncbi.nlm.nih.gov']
    custom_settings = {
        'ITEM_PIPELINES': {'__main__.CustomFilesPipeline': 1},
        # 'ITEM_PIPELINES': {'scrapy.pipelines.files.FilesPipeline': 1},
        'FILES_STORE': str( PATH_GENOMIC_FNA ),
    }

    start_urls = list(urls_archaea_organisms)  # Assuming this is defined elsewhere

    def parse(self, response):
        url_aav = urljoin(response.url, 'all_assembly_versions/')
        
        yield scrapy.Request(
            url=url_aav,
            callback=self.parse_through,
            meta={
                'base_url': response.url,
                'organism': response.url.split('/')[-2]
                }
        )

        
    def parse_through(self, response):
        # avs =: assembly_versions
        hrefs_avs = response.xpath('//pre/a/@href').getall()
        hrefs_avs = hrefs_avs[1:]
        href_1 = hrefs_avs[0]

        if ( href_1 == "suppressed/" ):
            yield scrapy.Request(
                url= urljoin(response.url, href_1),
                callback=self.parse_through,
                meta={
                    'base_url': response.url,
                    'organism': response.meta['organism']
                    }
            )
        else:
            yield scrapy.Request(
                url= urljoin(response.url, href_1),
                callback=self.download_files,
                meta={
                    'base_url': response.url,
                    'organism': response.meta['organism'],
                    'genome_version': href_1.split('/')[-2]
                    }
            )

        
    def download_files(self, response):
        genome_file_name = str(response.meta['genome_version']) + '_genomic.fna.gz'
        urls_download_genome = [urljoin(response.url, genome_file_name)]

        item = genomeNameItem()
        item['file_urls'] = urls_download_genome
        item['genome_file_name'] = str(response.meta['organism']).lower() \
            + '/' + str(genome_file_name)
        
        print("Downloading genome for: ", response.meta['organism'])

        yield item

In [22]:
print(list(urls_archaea_organisms[:5]))
print(urls_archaea_organisms[:5])

['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/', 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_infernus/']
['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/ANME-2_cluster_archaeon_HR1/'
 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/'
 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_brierleyi/'
 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/'
 'https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_infernus/']


### Crawler

In [24]:
len(urls_archaea_organisms)

1237

In [23]:
process = CrawlerProcess()
process.crawl(SpiderGenomesArchaea)
process.start()

# last runtime (2024-03-19): 2m 54.6s
# 1193 genomes downloaded from 1237 links

2024-03-19 14:30:17 [scrapy.utils.log] INFO: Scrapy 2.10.0 started (bot: scrapybot)
2024-03-19 14:30:17 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.10.4, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 22.10.0, Python 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 08:57:19) [GCC 11.3.0], pyOpenSSL 23.1.1 (OpenSSL 3.1.2 1 Aug 2023), cryptography 40.0.2, Platform Linux-6.5.0-26-generic-x86_64-with-glibc2.35
2024-03-19 14:30:17 [scrapy.addons] INFO: Enabled addons:
[]
2024-03-19 14:30:17 [scrapy.crawler] INFO: Overridden settings:
{}
2024-03-19 14:30:17 [py.warnings] WARNING: /home/diegoflm/mambaforge/envs/web_scraping/lib/python3.11/site-packages/scrapy/utils/request.py:248: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so fo

2024-03-19 14:30:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidiplasma_sp._MBA-1/all_assembly_versions/GCF_000949015.1_ASM94901v1/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidiplasma_sp._MBA-1/all_assembly_versions/)
2024-03-19 14:30:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Aigarchaeota_archaeon_JGI_0000001-A7/all_assembly_versions/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Aigarchaeota_archaeon_JGI_0000001-A7/)
2024-03-19 14:30:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Aigarchaeota_archaeon_SCGC_AAA471-D15/> (referer: None)
2024-03-19 14:30:19 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Aigarchaeota_archaeon_SCGC_AAA471-B22/all_assembly_versions/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/A

2024-03-19 14:30:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Actinarchaeum_halophilum/all_assembly_versions/GCF_027563145.1_ASM2756314v1/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Actinarchaeum_halophilum/all_assembly_versions/)
2024-03-19 14:30:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Aigarchaeota_archaeon_JGI_0000001-H6/all_assembly_versions/suppressed/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Aigarchaeota_archaeon_JGI_0000001-H6/all_assembly_versions/)


2024-03-19 14:30:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/all_assembly_versions/GCF_009428885.1_ASM942888v1/GCF_009428885.1_ASM942888v1_genomic.fna.gz> (referer: None)
2024-03-19 14:30:20 [scrapy.pipelines.files] DEBUG: File (downloaded): Downloaded file from <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_ambivalens/all_assembly_versions/GCF_009428885.1_ASM942888v1/GCF_009428885.1_ASM942888v1_genomic.fna.gz> referred in <None>
2024-03-19 14:30:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/all_assembly_versions/GCF_000213215.1_ASM21321v1/GCF_000213215.1_ASM21321v1_genomic.fna.gz> (referer: None)
2024-03-19 14:30:20 [scrapy.pipelines.files] DEBUG: File (downloaded): Downloaded file from <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Acidianus_hospitalis/all_assembly_versions/GCF_000213215.1_ASM21321v1/

2024-03-19 14:30:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Actinarchaeum_halophilum/all_assembly_versions/GCF_027563145.1_ASM2756314v1/>
{'file_urls': ['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Actinarchaeum_halophilum/all_assembly_versions/GCF_027563145.1_ASM2756314v1/GCF_027563145.1_ASM2756314v1_genomic.fna.gz'],
 'genome_file_name': 'actinarchaeum_halophilum/GCF_027563145.1_ASM2756314v1_genomic.fna.gz'}
2024-03-19 14:30:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Archaeoglobus_fulgidus/> (referer: None)
2024-03-19 14:30:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Archaeoglobus_neptunius/> (referer: None)
2024-03-19 14:30:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/Archaeoglobus_sp./> (referer: None)
2024-03-19 14:30:21 [scrapy.core.engine] DE

...

2024-03-19 14:33:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR67/all_assembly_versions/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR67/)
2024-03-19 14:33:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/all_assembly_versions/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/)
2024-03-19 14:33:09 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Nitrososphaera_sp./all_assembly_versions/GCF_946478925.1_SRR10754057_bin.7_metawrap_v1.3.0_MAG/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Nitrososphaera_sp./all_assembly_versions/)
2024-03-19 14:33:09 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_M

2024-03-19 14:33:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanosphaera_sp./all_assembly_versions/GCF_902494985.1_UHGG-TPA_MGYG-HGUT-04410/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanosphaera_sp./all_assembly_versions/)
2024-03-19 14:33:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/all_assembly_versions/suppressed/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR60/all_assembly_versions/)
2024-03-19 14:33:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HB70/all_assembly_versions/suppressed/GCF_000496195.1_A07HB70_072612b_select/> (referer: https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HB70/all_assembly_versions/suppressed/)
2024-03-19 14:33:10 [scrapy.

2024-03-19 14:33:10 [scrapy.core.engine] DEBUG: Crawled (404) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HB70/all_assembly_versions/suppressed/GCF_000496195.1_A07HB70_072612b_select/GCF_000496195.1_A07HB70_072612b_select_genomic.fna.gz> (referer: None)
2024-03-19 14:33:10 [scrapy.pipelines.files] WARNING: File (code: 404): Error downloading file from <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HB70/all_assembly_versions/suppressed/GCF_000496195.1_A07HB70_072612b_select/GCF_000496195.1_A07HB70_072612b_select_genomic.fna.gz> referred in <None>
2024-03-19 14:33:10 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanolobus_sp./all_assembly_versions/GCF_963665675.1_piMetPara1.Methanolobus_sp_1.1/GCF_963665675.1_piMetPara1.Methanolobus_sp_1.1_genomic.fna.gz> (referer: None)
2024-03-19 14:33:10 [scrapy.pipelines.files] DEBUG: File (downloaded): Downloaded fi

2024-03-19 14:33:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HN63/all_assembly_versions/suppressed/GCF_000496215.1_A07HN63_021313_select/>
{'file_urls': ['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HN63/all_assembly_versions/suppressed/GCF_000496215.1_A07HN63_021313_select/GCF_000496215.1_A07HN63_021313_select_genomic.fna.gz'],
 'genome_file_name': 'uncultured_archaeon_a07hn63/GCF_000496215.1_A07HN63_021313_select_genomic.fna.gz'}
2024-03-19 14:33:10 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR67/all_assembly_versions/suppressed/GCF_000496175.1_A07HR67_11212a_select/>
{'file_urls': ['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_archaeon_A07HR67/all_assembly_versions/suppressed/GCF_000496175.1_A07HR67_11212a_select/GCF_000496175.1_A07HR67_11212a_select_genomic.fna.gz'],
 'genome_

2024-03-19 14:33:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanospirillum_sp./all_assembly_versions/GCF_963668415.1_piHetPala1.Methanospirillum_sp_1.1/GCF_963668415.1_piHetPala1.Methanospirillum_sp_1.1_genomic.fna.gz> (referer: None)
2024-03-19 14:33:12 [scrapy.pipelines.files] DEBUG: File (downloaded): Downloaded file from <GET https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanospirillum_sp./all_assembly_versions/GCF_963668415.1_piHetPala1.Methanospirillum_sp_1.1/GCF_963668415.1_piHetPala1.Methanospirillum_sp_1.1_genomic.fna.gz> referred in <None>
2024-03-19 14:33:12 [scrapy.core.scraper] DEBUG: Scraped from <200 https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanospirillum_sp./all_assembly_versions/GCF_963668415.1_piHetPala1.Methanospirillum_sp_1.1/>
{'file_urls': ['https://ftp.ncbi.nlm.nih.gov/genomes/refseq/archaea/uncultured_Methanospirillum_sp./all_assembly_versions/GCF_9